# 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.getcwd()), ".."))


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from certainty import load_file

2025-05-05 08:05:52.052223: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-05 08:05:52.387566: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-05 08:05:53.519947: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [88]:
val = load_file("en_test.json")

In [97]:
results = []
for i in range(5):
    with open("../results/eventgraph/run" + str(i+1) +"/prediction_test.json") as f:
        data = json.load(f)
    data_id = {v['sent_id']:v for v in data}
    samples = []


    for v in val:
        sent_id = v['sent_id']
        pred = data_id[sent_id]
        triggers = pred['triggers']
        
        d = {}
        for obj in triggers:
            trigger = obj['trigger'][0][0]
            span = obj['trigger'][1][0]
            factuality = obj['modality']
            d[trigger+span] = factuality
    
        d2 = {}
        for ev in v['events']:
            trigger = ev['trigger'][0][0]
            span = ev['trigger'][1][0]
            factuality = ev['event_modality']
            d2[trigger+span] = factuality
            
            if (trigger+span) in d:
                samples.append(
                    {
                        'true': factuality,
                        'pred': d[(trigger+span)],
                        'label': 'discovered',
                        'polarity': ev['event_polarity'],
                        'genericity': ev['event_genericity'],
                        'type': ev['event_type'],
                        'text': v['text'],
                    }
                )
            else:
                samples.append(
                    {
                        'true': factuality,
                        'pred': "Other" if factuality == 'Asserted' else 'Asserted',
                        'label': 'undiscovered',
                        'polarity': ev['event_polarity'],
                        'genericity': ev['event_genericity'],
                        'type': ev['event_type'],
                        'text': v['text'],
                    }
                )
                
        for ev in triggers:
            trigger = obj['trigger'][0][0]
            span = obj['trigger'][1][0]
            factuality = obj['modality']
            if (trigger+span) not in d2:
                samples.append({
                    'true': 'Other' if factuality == 'Asserted' else 'Asserted',
                    'pred': factuality,
                    'label': 'spurious',
                    'text': v['text']
                })
                
    df = pd.DataFrame(samples)
    df.to_csv('../results/eventgraph/'+str(i+1)+'.csv')
    
    res = {}
    fp = len(df[df['label'] == 'spurious'])
    fn = len(df[df['label'] == 'undiscovered'])
    tp = len(df[df['label'] == 'discovered'])
    t_precision = tp/(tp+fp) if tp != 0 else 0
    t_recall = tp/(tp+fn) if tp != 0 else 0
    t_f1 = (2*t_precision*t_recall)/(t_precision+t_recall) if (t_precision + t_recall) != 0 else 0
    res['eval/trigger_precision'] = t_precision
    res['eval/trigger_recall'] = t_recall
    res['eval/trigger_f1'] = t_f1
    do_tp = len(df[(df['label'] == 'discovered') & (df['true'] == 'Other') & (df['pred'] == 'Other')])
    do_fp = len(df[(df['label'] == 'discovered') & (df['true'] == 'Asserted') & (df['pred'] == 'Other')])
    do_fn = len(df[(df['label'] == 'discovered') & (df['true'] == 'Other') & (df['pred'] == 'Asserted')])
    do_precision = do_tp/(do_tp+do_fp) if do_tp != 0 else 0
    do_recall = do_tp/(do_tp+do_fn) if do_tp != 0 else 0
    do_f1 = (2*do_precision*do_recall)/(do_precision+do_recall) if (do_precision+do_recall) != 0 else 0
    res['eval/discovered_other_precision'] = do_precision
    res['eval/discovered_other_recall'] = do_recall
    res['eval/discovered_other_f1'] = do_f1
    da_tp = len(df[(df['label'] == 'discovered') & (df['true'] == 'Asserted') & (df['pred'] == 'Asserted')])
    da_fp = len(df[(df['label'] == 'discovered') & (df['true'] == 'Other') & (df['pred'] == 'Asserted')])
    da_fn = len(df[(df['label'] == 'discovered') & (df['true'] == 'Asserted') & (df['pred'] == 'Other')])
    da_precision = da_tp/(da_tp+da_fp) if da_tp != 0 else 0
    da_recall = da_tp/(da_tp+da_fn) if da_tp != 0 else 0
    da_f1 = (2*da_precision*da_recall)/(da_precision+da_recall) if (da_precision+da_recall) != 0 else 0
    res['eval/discovered_asserted_precision'] = da_precision
    res['eval/discovered_asserted_recall'] = da_recall
    res['eval/discovered_asserted_f1'] = da_f1
    a_tp = len(df[(df['true'] == 'Asserted') & (df['pred'] == 'Asserted')])
    a_fp = len(df[(df['true'] == 'Other') & (df['pred'] == 'Asserted')])
    a_fn = len(df[(df['true'] == 'Asserted') & (df['pred'] == 'Other')])
    a_precision = a_tp/(a_tp+a_fp) if a_tp != 0 else 0
    a_recall = a_tp/(a_tp+a_fn) if a_tp != 0 else 0
    a_f1 = (2*a_precision*a_recall)/(a_precision+a_recall) if (a_precision+a_recall) != 0 else 0
    
    o_tp = len(df[(df['true'] == 'Other') & (df['pred'] == 'Other')])
    o_fp = len(df[(df['true'] == 'Asserted') & (df['pred'] == 'Other')])
    o_fn = len(df[(df['true'] == 'Other') & (df['pred'] == 'Asserted')])
    o_precision = o_tp/(o_tp+o_fp) if o_tp != 0 else 0
    o_recall = o_tp/(o_tp+o_fn) if o_tp != 0 else 0
    o_f1 = (2*o_precision*o_recall)/(o_precision+o_recall) if (o_precision+o_recall) != 0 else 0
    res['eval/asserted_precision'] = a_precision
    res['eval/asserted_recall'] = a_recall
    res['eval/asserted_f1'] = a_f1
    res['eval/other_precision'] = o_precision
    res['eval/other_recall'] = o_recall
    res['eval/other_f1'] = o_f1
    results.append(res)
        
    


In [98]:
df = pd.DataFrame(results)


In [99]:
res

{'eval/trigger_precision': 0.4376278118609407,
 'eval/trigger_recall': 0.5047169811320755,
 'eval/trigger_f1': 0.4687842278203724,
 'eval/discovered_other_precision': 0.8461538461538461,
 'eval/discovered_other_recall': 0.673469387755102,
 'eval/discovered_other_f1': 0.75,
 'eval/discovered_asserted_precision': 0.9085714285714286,
 'eval/discovered_asserted_recall': 0.9636363636363636,
 'eval/discovered_asserted_f1': 0.9352941176470588,
 'eval/asserted_precision': 0.3404710920770878,
 'eval/asserted_recall': 0.4441340782122905,
 'eval/asserted_f1': 0.38545454545454544,
 'eval/other_precision': 0.14224137931034483,
 'eval/other_recall': 0.0967741935483871,
 'eval/other_f1': 0.11518324607329843}

In [91]:
df.to_csv('../results/end_to_end_results/test_eventgraph_results.csv')

In [85]:
df.describe()

,eval/trigger_precision,eval/trigger_recall,eval/trigger_f1,eval/discovered_other_precision,eval/discovered_other_recall,eval/discovered_other_f1,eval/discovered_asserted_precision,eval/discovered_asserted_recall,eval/discovered_asserted_f1,eval/asserted_precision,eval/asserted_recall,eval/asserted_f1,eval/other_precision,eval/other_recall,eval/other_f1
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.444860,0.478416,0.457559,0.793864,0.728320,0.758582,0.904088,0.931851,0.917612,0.333256,0.378042,0.353355,0.147736,0.125171,0.134869
std,0.007772,0.080954,0.038470,0.045227,0.095367,0.070992,0.031199,0.008541,0.019365,0.012813,0.053033,0.031463,0.019172,0.013282,0.012370
min,0.436482,0.342574,0.391846,0.727273,0.606061,0.661157,0.869347,0.920213,0.894057,0.313830,0.287105,0.299873,0.122754,0.102302,0.115774
25%,0.442834,0.471287,0.457253,0.787879,0.666667,0.727273,0.876190,0.930000,0.904177,0.330153,0.376744,0.353326,0.133333,0.128686,0.129542
50%,0.443281,0.502970,0.471243,0.800000,0.750000,0.774194,0.910112,0.931034,0.920455,0.332649,0.399538,0.361546,0.151899,0.128713,0.139332
75%,0.444030,0.530693,0.478999,0.800000,0.764706,0.776119,0.920792,0.934010,0.925373,0.343925,0.410714,0.374364,0.164557,0.129032,0.144645
max,0.457672,0.544554,0.488455,0.854167,0.854167,0.854167,0.944000,0.944000,0.944000,0.345725,0.416107,0.377665,0.166134,0.137124,0.145049
